# 레시피 추천 시스템

https://chan-chance.tistory.com/22

In [1]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from gensim.models import Word2Vec

In [2]:
coupang = pd.read_csv('../../data/coupang_filtering_pre.csv', index_col=0)

#가중치까지 계산한 df을 불러옴
ckg = pd.read_csv('../../data/ckg_weighted.csv', index_col=0)

#word2vec 사용을 위한 데이터 포멧 변경
ckg['요리재료_전처리'] = ckg['요리재료_전처리'].apply(lambda x : x.split(', ')) # 1차원 list 형태
ckg.head()

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,알레르기,weighted_clipping
0,131871,★현미호두죽,현미호두죽,2912,0,9,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,"[현미, 찹쌀, 호두, 물, 소금]",견과류,314.655903
1,139247,부들부들 보들보들 북어갈비♥,북어갈비,6865,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,"[북어포, 찹쌀가루, 간장, 설탕, 물, 다진파, 다진마늘, 참기름, 깨소금, 후춧가루]","콩, 견과류",284.844463
2,149207,토마토스파게티♥,토마토스파게티,12754,2,36,볶음,일상,가공식품류,면/만두,한번 만들어 두면 이모저모 쓸 용도가 많은 토마토소스를 이용한 토마토 스파게티.,[재료] 파스타면 [양념] 토마토 1개| 토마토 페이스트 3T| 양파 1/2개| 다...,"[파스타면, 토마토, 토마토, 양파, 다진마늘, 피망, 올리브유]","밀가루, 토마토",212.802388
3,151148,표고버섯탕수,표고버섯탕수,16053,2,194,튀김,손님접대,버섯류,메인반찬,탕수육 먹을때 칼로리때문에 걱정하시는 분들!! 이제는 걱걱걱 정하지 마세요~ 칼로리...,[재료] 건표고버섯 9개| 오이 1/2개| 당근 1/2개| 양파 1/2개| 사과 1...,"[건표고버섯, 오이, 당근, 양파, 사과, 그외의, 과일, 녹말가루, 물, 계란 노...","난류, 콩",279.114241
4,153040,달콤한 마늘향이 가득해~ 갈릭치킨♥,갈릭치킨,8148,0,38,튀김,손님접대,닭고기,메인반찬,요즘 갈릭이 대세인거 같아요~ 그래서 오늘은 제가 너무 좋아하는 닭과 마늘을 한번에...,[재료] 닭 1마리 [닭양념] 통마늘 3개| 다진마늘 1T| 화이트와인또는우유| 올...,"[닭, 통마늘, 다진마늘, 화이트와인또는우유, 올리브유또는버터, 마늘솔트또는소금, ...","우유, 닭",251.350570


In [223]:
# coupang[coupang.상세카테고리 == '']
coupang[coupang.상품.str.contains('전')]

,Unnamed: 0.1,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,상품_전처리
0,0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,1%,25900,"(100g당 1,625원)",4.5,11438,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎...",전골,프레시지 대한곱창 곱창전골
4,4,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",14900,0,14900,"(100g당 1,717원)",5.0,37028,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스",전골,프레시지 더큰 햄 가득 부대전골
9,9,1,188130036,537230639,4402737129,프레시지 서울식 불고기 전골,"프레시지 서울식 불고기 전골, 424.5g, 1개",12900,15%,10900,"(100g당 2,568원)",4.5,8913,0,"소고기, 당면, 팽이버섯, 소스, 참기름 등으로 구성되어 있습니다.","소고기,당면,팽이버섯,소스,참기름",전골,프레시지 서울식 불고기 전골
20,20,1,6168220034,12021424122,79293740157,청정원 호밍스 낙곱새전골 (냉동),"청정원 호밍스 낙곱새전골 (냉동), 800g, 1개",13500,0,13500,"(100g당 1,688원)",4.5,626,0,"낙곱새전골 소스, 낙지곱창, 우동면으로 구성되어 있습니다.",우동면,전골,청정원 호밍스 낙곱새전골 ( 냉동 )
26,26,1,6636019641,15150790746,82372315487,애슐리 소불고기 버섯전골,"애슐리 소불고기 버섯전골, 630g, 1개",17310,1%,17010,"(100g당 2,700원)",4.0,36,0,"소고기 포장육, 팽이버섯, 느타리버섯, 표고버섯, 양파, 배추, 청경채, 건면, 소...","팽이버섯,느타리버섯,표고버섯,양파,배추,청경채,건면",전골,애슐리 소불고기 버섯전골
42,42,1,6579540098,14800639051,82040415828,쿡킷 해물 짬뽕 비비고 만두전골 2인분 (냉동),"쿡킷 해물 짬뽕 비비고 만두전골 2인분 (냉동), 1개, 824g",15210,0,15210,"(100g당 1,846원)",4.5,476,0,"김치 왕만두, 중화면, 혼합 채소(청경채, 목이버섯), 혼합 해물(오징어, 자숙홍합...","김치 왕만두,중화면,청경채,목이버섯,오징어,소스",전골,쿡킷 해물 짬뽕 비비 고 만두전골 2인분 ( 냉동 )
43,43,1,6579540231,14800639656,82040416529,쿡킷 얼큰 버섯 비비고 만두전골 2인분 (냉동),"쿡킷 얼큰 버섯 비비고 만두전골 2인분 (냉동), 850g, 1개",15900,5%,15100,"(100g당 1,776원)",4.5,919,0,"왕만두, 소스, 칼국수 면, 혼합 채소(느타리버섯, 대파)로 구성되어 있습니다.","왕만두,소스,칼국수면,느타리버섯,대파",전골,쿡킷 얼큰 버섯 비비 고 만두전골 2인분 ( 냉동 )
47,47,1,6608252642,14972855103,82196037673,쿡킷 육수가득 소불고기 전골 2인분 (냉동),"쿡킷 육수가득 소불고기 전골 2인분 (냉동), 1개, 610g",16900,10%,15210,"(100g당 2,493원)",4.5,496,0,"잡채 면, 쇠고기, 대파, 팽이버섯, 간장 배 양념으로 구성되어 있습니다.","쇠고기,대파,팽이버섯",전골,쿡킷 육수 가득 소불고기 전골 2인분 ( 냉동 )
48,48,1,6661336303,15292003271,82512414231,푸드어셈블 채선당 불고기 꽃만두 전골 & 칼국수 (냉동),"푸드어셈블 채선당 불고기 꽃만두 전골 & 칼국수 (냉동), 1030g, 1팩",17900,2%,17400,"(100g당 1,689원)",4.5,32,0,"만두, 알배추, 소스, 양념육 등으로 구성되어 있습니다.","만두,알배추,소스,양념육",전골,푸드어셈블 채선당 불고기 꽃 만두 전골 & 칼국수 ( 냉동 )
54,54,1,4841176792,6256834516,73552540510,곰곰 샤브 버섯 국수 전골 (냉장),"곰곰 샤브 버섯 국수 전골 (냉장), 1.01kg, 1개",12900,10%,11490,"(100g당 1,138원)",4.5,1687,0,"쇠고기, 배추, 생면, 칼국수면, 소간장소스, 대파, 팽이버섯, 느타리버섯, 표고버...","쇠고기,배추,생면,칼국수면,소간장소스,대파,팽이버섯,느타리버섯,표고버섯,샤브용육수",전골,곰곰 샤브 버섯 국수 전골 ( 냉장 )


In [141]:
a = ckg[ckg.요리명.str.contains('해장')]
a#[a.요리방법별명 == '찜']

,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,weighted_clipping
1477,1545709,아침국 해장국으로 좋은 시원하고 감칠맛나는 콩나물국 끓이기,콩나물해장국,20511,7,115,끓이기,일상,채소류,국/탕,아침국 해장국으로 좋은 시원하고 감칠맛나는 콩나물국입니다.,[재료] 콩나물| 양파(중) 1/4개| 쪽파 약간 | [멸치다시마 육수]| 멸치 1...,"[콩나물, 양파, 쪽파, 멸치, 다시마, 청주, 다진마늘, 국간장, 천일염]",220.142504
2084,1674596,콩나물 해장라면 만들기,콩나물해장라면,69113,8,289,끓이기,야식,채소류,면/만두,야식으로 먹었던 콩나물 라면이에요. 면에 콩나물 하나 넣은 것 뿐인데 기름기도 적어...,[재료] 라면 (소 84g) 2봉지| 콩나물 120g(취향껏)| 대파| 풋고추| 홍...,"[라면, 콩나물, 대파, 풋고추, 홍고추, 물]",303.953550
2193,1689906,속풀이 해장국으로 좋은 황태국,황태해장국,30066,17,280,끓이기,해장,건어물류,국/탕,시원한 속풀이 해장국으로 좋은 황태국입니다.,[재료] 황태 50g| 다시마육수 3큰술| 무우 50~70g| 계란 1개| 다진마늘...,"[황태, 다시마육수, 무우, 계란, 다진마늘, 대파, 홍고추, 다시마, 물, 국간장...",311.161371
2246,1697618,홈메이드 뜨끈한 뼈해장국,뼈해장국,55562,28,1334,끓이기,해장,육류,국/탕,뜨끈한 뚝배기 뼈다귀해장국 만들어 봤어요. 무작정 핏물부터 빼면서 인터넷 찾아봤는데...,[재료] 국내산 냉장 돼지등뼈 1.5kg| 우거지 200g| 대파 2뿌리| 양파 1...,"[국내산 냉장, 우거지, 대파, 양파, 된장, 청주, 후추, 고추가루, 된장, 청주...",1148.834385
3163,1826505,선지 해장국 끓이기,선지해장국,49680,20,221,끓이기,해장,육류,국/탕,선지해장국을 집에서 끓여보겠습니다.,[재료] 선지 2공기| 얼갈이 1/2단| 양 1장| 청양고추 5개| 대파 1+1/2...,"[선지, 얼갈이, 양, 청양고추, 대파, 양파, 콩나물, 사골, 육수, 된장, 다진...",255.005659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95185,6948805,무청시래기 뼈해장국~겨울 보양식,무청시래기뼈해장국,1435,0,25,끓이기,영양식,돼지고기,국/탕,예로부터 겨울먹거리 무청시래기에 돼지등뼈를 푹 끓여서 뜨끈하게 해장국 한그릇해보세요,[재료] 돼지등뼈 4~5kg| 무청 우거지 800g| 월계수잎 7~8장| 편생강 1...,"[돼지등뼈, 무청, 월계수잎, 편생강, 사과 작은것, 대파, 청양고추, 건 새송이버...",346.634614
95302,6948970,다슬기국 올갱이해장국 보약 한그릇 집에서,올갱이해장국,2982,0,30,끓이기,영양식,기타,국/탕,저는 경상도에서 자라서 어릴때는 할머니댁에 농사를 지으시니 주말마다 들어가서 놀았어...,[재료] 배추잎 6장| 부추 1줌| 파 1대| 청양고추 2개| 양파 1/2개| 된장...,"[배추잎, 부추, 파, 청양고추, 양파, 된장, 고추장, 간마늘, 올갱이, 육수티백...",317.172405
95327,6949005,감자탕 끓이는법 묵은지요리 돼지등뼈 해장국,돼지등뼈해장국,1640,0,25,끓이기,영양식,돼지고기,국/탕,뜨끈한 국물이 최고인 감자탕 맛있게 만들어 볼께요.,[재료] 돼지등뼈 1.5kg| 설탕 1/3컵| 간마늘1T| 된장1T| 고춧가루2T|...,"[돼지등뼈, 설탕, 간마늘, 된장, 고춧가루, 설탕, 청양고추, 파, 배추김치, 된장]",342.250719
96179,6950175,#소위요리 #양평해장국만들기 #삶아낸 소위와 양지를 넣고 시원하게 끓여낸 양평해장국,양평해장국,3740,0,14,끓이기,해장,소고기,국/탕,양평해장국의 특징이 선지가 들어가지 않는 것이지요. 물론 선지를 넣어 주는 곳도 ...,[재료] 소위 600g| 한우양지 500g| 통마늘 10개| 대파 4대| 콩나물 1...,"[소위, 한우양지, 통마늘, 대파, 콩나물, 무, 만능양념장, 고춧가루, 다진마늘,...",300.981517


# 사전 정의

# word2vec 기반
- tfidf와 비교했을 때 w2v의 코사인이 카테고리 분류하지 않아도 유사한 레시피를 잘 찾아줌
- w2v에서 CBOW와 Skip-Gram 사이의 정확도는 정확한 데이터가 있어야 비교 가능할 듯

분리해야하는 세부카테고리 : 탕, 순대, 한식

In [3]:
#만개의 레시피 카테고리 사전
ckg_dict = {
    '요리방법별명' : ['끓이기', '굽기', '볶음', '튀김', '기타', '부침', '조림', '찜', '무침', '절임', '비빔', '데치기', '삶기', '회'],
    '요리상황별명' : ['일상', '술안주', '손님접대', '도시락', '야식', '간식', '영양식', '명절', '초스피드','다이어트', '해장', '이유식'],
    '요리재료별명' : ['쌀', '건어물류', '가공식품류', '버섯류', '닭고기', '해물류', '채소류', '육류', '밀가루','콩/견과류', '돼지고기', '달걀/유제품', '소고기', '곡류', '과일류'],
    '요리종류별명' : ['밥/죽/떡', '메인반찬', '면/만두', '양식', '밑반찬', '찌개', '국/탕', '김치/젓갈/장류', '퓨전']
}

# 만개의레시피의 카테고리를 기준으로 쿠팡의 세부 카테고리를 분류
coupang_dict = {
    '전골':'국/탕', 
    '생선탕':'해물류',
    '찌개/짜글이':'찌개', 
    '한식':'메인반찬,굽기', 
    '알탕':'해물류',
    '해물탕':'국/탕', 
    '기타':'null', 
    '국':'국/탕', 
    '씨푸드':'해물류', 
    '구이':'굽기',
    '찜':'찜', 
    '닭':'닭고기', 
    '면':'면/만두', 
    '국수':'면/만두', 
    '스테이크':'굽기,양식', 
    '파스타':'면/만두,양식', 
    '감바스':'양식', 
    '볶음':'볶음', 
    '떡볶이':'밥/죽/떡',
    '마파두부':'볶음', 
    '마라탕':'국/탕', 
    '쌈':'메인반찬', 
    '무침':'무침', 
    '샌드위치':'일상,기타', 
    '족발':'돼지고기', 
    '리조또':'밥/죽/떡', 
    '카레':'일상',
    '덮밥/비빔밥':'밥/죽/떡'
}

# 양념 재료 사전
sauce_dict = ['가루','설탕','간장','된장','식초','오일','액젓','소금','물','소주','스프','고추장','고춧가루','사이다','소스','케첩','물엿','올리고당','매실청','식용유','후추','젓갈','즙','정종','와인','맛술']


In [4]:
##### 요리재료와 쿠팡 구성정보 일치 #####
def get_ingredients_pre(word):
    if ('소고기' in word)or('쇠고기' in word): word = '소고기'
    elif ('건면' in word)or('생면' in word)or('파스타 면' in word)or('파스타면' in word)or('라면 사리' in word)or('우동 사리' in word)or('라면사리' in word)or('우동사리' in word)or('칼국수 면' in word)or('칼국수면' in word)or('스파게티 면' in word)or('스파게티면' in word)or('페투치네' in word)or('우동면' in word)or('우동 면' in word)or('숙면' in word)or('소면' in word)or('쫄면' in word):word = '면'
    elif '고추장' in word: word = '고추장'
    elif ('고추' in word) and ('고추장' not in word): word = '고추'
    elif '새우젓' in word: word = '새우젓'
    elif ('새우' in word)and('새우젓' not in word): word = '새우'
    #elif '명태알 소스' in word: word = '명란'
    elif ('닭' in word)or('수비드 찜닭' in word): word = '닭'
    elif ('계란' in word)or('달걀' in word): word = '계란'
    elif ('소금' in word)or('천일염' in word): word = '소금'
    elif '다진' == word: word = '다진마늘'
    elif ('케첩' in word)or('케찹' in word): word = '케첩'
    elif ('식용유' in word)or('올리브유' in word)or('카놀라유' in word)or('기름' in word): word = '식용유'
    else:
        #단어 리스트에 있는 재료명 통일
        word_list = ['아귀','매생이','꼬막','가루','어묵','비엔나 소시지','대파','피망','파프리카','무우','양파','미역'
                     ,'오이','식초','간장','오일','스프','아보카도','물엿','올리고당','후추','젓갈','즙','와인','정종','액젓']
        for num in range(len(word_list)):
            if word_list[num] in word: 
                word = word_list[num]
    return word


##### input 값 전처리 후 list에 단어별 저장 #####
def get_product_list(num, product_df):
    # 상품명 전처리
    product = product_df.at[num, '상품_전처리'] #string으로 반환
    product_list = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\(\)]', '', product).split(' ') #특수문자제거(만개의레시피 데이터엔 특수문자가 없어서 인식 불가)
    product_list = list(filter(None, product_list)) #null 제거(안 할 경우 레시피 전체가 선택되어 작동 불가)
    
    #구성정보 전처리
    str_list = product_df.at[num, '구성정보_전처리']
    ingredients_list = str_list.split(',')
    
    #구성정보 요리재료명 통일
    for idx in range(len(ingredients_list)):
        ingredients_list[idx] = get_ingredients_pre(ingredients_list[idx])
    
    #구성정보 요리재료의 중복값 제거   
    result = []
    [result.append(x) for x in ingredients_list if x not in result]
    
    return product_list, result #쿠팡의 상품명, 구성정보_전처리의 요리재료명 반환


##### input 상품의 상세카테고리 추출 #####
def get_product_category(num, product_df):
    product_category = coupang_dict[coupang.at[num, '상세카테고리']].split(',') #리스트 반환
    ckg_key_value = {}
    if product_category == ['null']: #쿠팡의 상세카테고리==기타인 경우, null 반환
        ckg_key_value = {'null': 'null'}
    else:
        for category in product_category:
            for ckg_key in ckg_dict.keys(): #만개의레시피 해당 컬럼의 상세카테고리 리스트 저장
                ckg_value = ckg_dict.get(ckg_key) 
                if category in ckg_value: #만개의레시피 상세카테고리에 쿠팡 상세카테고리가 있으면
                    ckg_key_value[ckg_key] = category # { key=만개의레시피 해당컬럼 : value=해당컬럼의 상세 카테고리}로 저장
    return ckg_key_value


##### input 상품과 동일한 요리명 및 카테고리 의 레시피만 추출 #####
def get_recipe_df(product_list, ckg_key_value):
    #1차필터링
    recipe_df = pd.DataFrame() #추출한 만개의 레시피를 저장할 빈 df 생성
    for word in product_list:
        # 쿠팡상품명과 레시피요리명을 동일하게 변경
        if '보일링' in word: word = '해물찜'
        elif '씨 키트' in word: word = '해물'
        elif '냉동' in word: word = 'NaN' #쿠팡 상품명에 냉동이 있는 경우 제외하기 위한 값
        else: pass
        #해당 쿠팡 상품명 단어와 일치하는 요리명의 index만 저장
        recipe = ckg[ckg['요리명'].str.contains(word)] 
        recipe_df = pd.concat([recipe_df, recipe]) #위에서 생성한 df와 결합
    #2차 필터링
    recipe_df_filter = pd.DataFrame()
    for ckg_key in ckg_key_value.keys():
        if ckg_key == 'null': 
            #상세카테고리==기타 인 경우, 위의 df 그대로 사용 
            recipe_df_filter = recipe_df
        else:
            #해당하는 컬럼의 일치하는 상세카테고리만 저장
            recipe_df_key = recipe_df[recipe_df[f'{ckg_key}'] == ckg_key_value.get(ckg_key)]
            recipe_df_filter = pd.concat([recipe_df_filter, recipe_df_key])     
    #한번에 중복값 제거
    recipe_df_filter.drop_duplicates(subset=['레시피일련번호'], keep='first', inplace=True) 
    return recipe_df_filter


##### 추출한 만개의 레시피의 요리재료명 통일 및 word2vec을 위한 이중리스트 생성 #####
def get_recipe_pre(recipe_index, recipe_df):
    recipe_lists = []
    for index in recipe_index:
        man_ingredients = recipe_df.loc[index, '요리재료_전처리']
        # 요리재료가 word2vec을 위해 list형태로 저장
        # 반복문으로 재료명 통일
        recipe_list = []
        for man_ingredient in man_ingredients:
            man_ingredient = get_ingredients_pre(man_ingredient)
            # 추가할 필요가 없는 양념재료 제거
            # 양념 사전와 불일치하는 경우에만 recipe_list에 추가
            if man_ingredient in sauce_dict:
                pass
            else:
                recipe_list.append(man_ingredient)
        # 이중 리스트로 '요리재료_전처리' 전체 반환
        recipe_lists.append(recipe_list)
    return recipe_lists


###### 단어 벡터 평균구하기 ######
def vectors(embedding, embedding_model):
    word_embedding=[]
    dec2vec=None
    count=0
    for word in embedding:
        try : 
            # word에 해당하는 단어를 워드투백에서 찾아 리스트에 붙힘
            # 100차원으로 형성됨
            word_embedding.append(embedding_model.wv[word]) 
            count+=1
        except : continue

    word_embedding2=np.array(word_embedding)
    v=word_embedding2.sum(axis=0)  
    
    if type(v)!=np.ndarray:
        return np.zeros(100)
    
    return(v/count)


###### 최종 알고리즘 ######
def wv_mealkit(sim,mealkit_index,top_n,recipe_df):
    # 코사인유사도를 구한 행렬을 역순으로 정렬 -> 유사도가 높은 순의 인덱스로 정렬됨 
    sim_sorted_ind=sim.argsort()[:,::-1]

    # sim_indexes는 이중리스트 상태이므로 이중리스트를 해제한 후 인덱스를 이용해 해당 내용 추출
    similar_indexes=sim_sorted_ind[mealkit_index,1:(top_n+1)]
    recipe = recipe_df.iloc[similar_indexes]  #.sort_values(by='weighted_clipping',ascending=False) # 정렬은 우선 제외
    return recipe


###### 레시피 only 재료 추출 ######
def get_recipe_only(ingredients_list, sim_mealkit_df):
    recipe_ingredient = []
    for idx in range(len(sim_mealkit_df)):
        value_list = sim_mealkit_df.요리재료_전처리.values[idx]
        
        value_set = [] #중복제거
        [value_set.append(x) for x in value_list if x not in value_set]
        
        for ingredient in ingredients_list:
            # input 구성정보와 요리재료가 일치하면 제외
            if ingredient in value_set:
                value_set.remove(ingredient)
        #레시피에만 있는 요리재료 추가
        recipe_ingredient.append(value_set)
    return recipe_ingredient

## 실행코드
- 알레르기: '갑각류, 견과류, 난류, 메밀, 밀가루, 우유, 연체동물류, 콩, 닭, 돼지, 복숭아, 소, 토마토, 생선류'

In [5]:
num = 189
allergy_num = '갑각류'

In [6]:
try:
    # coupang에서 input 값의 df 추출
    product_df = coupang.iloc[num:num+1, :]
    display(product_df)

    # input 상품명 및 구성정보 전처리 후 list로 추출
    product_list, ingredients_list = get_product_list(num, product_df)
except: 
    print("입력하신 번호의 쿠팡상품이 없습니다.")
    print('쿠팡 상품 개수 :', len(coupang))

try:
    ## 전처리
    ckg_key_value = get_product_category(num, product_df) # input 상품의 만개의 레시피와 일치하는 카테고리만 추출 {dict}
    recipe_df = get_recipe_df(product_list, ckg_key_value) # input 상품명과 일치하는 레시피만 추출
    
    if len(recipe_df)==0: raise #일치하는 레시피가 아예 없는 경우 빈df 반환됨 == 에러발생
    
    recipe_df = recipe_df[~recipe_df['알레르기'].str.contains(allergy_num, na=False)] #알레르기 컬럼 추가
    recipe_index = recipe_df.index.tolist()
    word = get_recipe_pre(recipe_index, recipe_df)  # 일치하는 만개의 레시피 요리재료명 통일
    recipe_df['요리재료_전처리'] = word  # 통일한 요리재료명 업데이트
    
    ## word2vec 모델 적용
    word.append(ingredients_list) # word list의 가장 마지막에 input 구성정보 추가
    wv_model = Word2Vec(sentences=word, vector_size=100, window=20, min_count=1,  sg=1, batch_words=1000, alpha=0.03) # word2vec 모델 생성
    wordvec=[vectors(x, wv_model) for x in word] #레시피 별 벡터 평균 추출
    w2v_sim=cosine_similarity(wordvec,wordvec) # 코사인유사도 계산

    #레시피 추출
    mealkit_index = len(w2v_sim)-1 # 코사인유사도 행렬에서 input 상품의 인덱스 저장
    sim_mealkit_df = wv_mealkit(w2v_sim, mealkit_index, 5, recipe_df) #input 상품과 유사한 레시피 df 추출
    display(sim_mealkit_df) 

    #레시피에만 있는 재료 추출
    print('ingredients = ', ingredients_list)
    recipe_ingredients = get_recipe_only(ingredients_list, sim_mealkit_df)
    recipe_ingredients = list(filter(None, recipe_ingredients)) # input 구성정보와 요리재료가 모두 일치(빈리스트)하면 제외 
    pprint(recipe_ingredients)
    
except: 
    print("유사한 레시피가 없습니다.")

,Unnamed: 0,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,상품_전처리
189,192,4,6636134619,15151379986,82372899340,모두의맛집 샤오짠 탄탄면 (냉동),"모두의맛집 샤오짠 탄탄면 (냉동), 1개, 580g",8800,0,8800,"(100g당 1,517원)",4.0,8,1,"구성품(청경채, 돼지고기, 두반장 소스 등 함유), 생면으로 구성되어 있습니다.","청경채,돼지고기,생면",면,모두의맛집 샤오 짠 탄탄면 ( 냉동 )


,레시피일련번호,레시피제목,요리명,조회수,추천수,스크랩수,요리방법별명,요리상황별명,요리재료별명,요리종류별명,요리소개,요리재료내용,요리재료_전처리,알레르기,weighted_clipping
39812,6869504,(라면의 화려한 변신) 탄탄면 / 탄탄면 만들기,탄탄면,8275,5,120,끓이기,일상,가공식품류,면/만두,예전부터 탄탄면 맛이 너무 궁금했었습니다!! 그러다가 술 진탕 먹고 해장을 위해 편...,[필수 재료] 라면사리 2봉지| 사골육수| 돼지고기 다짐육 100g [선택 재료] ...,"[면, 사골육수, 돼지고기, 청경채, 숙주나물, 다진 파, 고추, 다진 마늘, 다진...","콩, 견과류, 우유, 밀가루, 돼지",281.405724
87687,6939036,탄탄면 도전해보세요 코다 하우스,탄탄면,1642,0,42,끓이기,해장,밀가루,면/만두,중국에서 먹던 탄탄면 그맛 그대로 입니다.,[재료] 칼국수면 150g| 두반장 4티스푼| 고추기름 1스푼| 청경채| 땅콩쨈 1...,"[면, 두반장, 고추, 청경채, 땅콩쨈, 소고기, 사리곰탕, 마늘, 대파]","콩, 밀가루, 소, 견과류",344.064238
498,938201,꽃만두를 내 마음대로 샤오마이,샤오마이,6568,10,78,찜,간식,육류,면/만두,샤오마이 꽃 모양의 위가 뚫린 만두 보신적 있으시죠? 보통 위가 뚫린 꽃모양의 만...,[재료] 돼지고기 200g| 새우 200g| 왕 만두피 12장| 작은 만두피 24장...,"[돼지고기, 새우, 왕 만두피, 작은 만두피, 양파, 다진 쪽파, 청주, 다진 마늘...","돼지, 갑각류, 연체동물류, 밀가루",281.388876
21822,6836366,탄탄면 만들기 이연복쉐프 탄탄면 따라하기 국물있는 탄탄면 만드는 방법 라면스프없이 ...,탄탄면,36016,20,204,끓이기,해장,가공식품류,면/만두,집밥요리 탄탄면 만들기 이연복쉐프 탄탄면 따라하기 국물있는 탄탄면 만드는 방법 라면...,[고명 재료] 돼지고기다짐육| 고춧가루| 마늘| 대파| 간장 [라면 재료] 라면사리...,"[돼지고기다짐육, 마늘, 대파, 면, 사골곰탕, 땅콩버터, 팽이버섯, 고추, 두반장...","콩, 견과류, 우유, 밀가루, 돼지",252.018413
53454,6891041,탄탄면 땅콩버터로 맛을 낸 면요리,탄탄면,7074,0,147,끓이기,일상,가공식품류,면/만두,사골국물을 사용해 진득하고 고소한 맛의 국물. 거기에 매콤칼칼한 돼지고기볶음. 아삭...,[재료] 사골국물 750ml| 다진 돼지고기 90g| 숙주 80g| 칼국수면 2인분...,"[사골국물, 다진마늘, 숙주, 면, 썰은 표고버섯, 계란, 땅콩, 썰은, 고추, 두...","연체동물류, 견과류, 콩, 난류, 우유, 밀가루",300.202277


ingredients =  ['청경채', '돼지고기', '면']
[['사골육수', '숙주나물', '다진 파', '고추', '다진 마늘', '두반장', '땅콩버터'],
 ['두반장', '고추', '땅콩쨈', '소고기', '사리곰탕', '마늘', '대파'],
 ['새우', '왕 만두피', '작은 만두피', '양파', '다진 쪽파', '청주', '다진 마늘', '굴 소스'],
 ['돼지고기다짐육', '마늘', '대파', '사골곰탕', '땅콩버터', '팽이버섯', '고추', '두반장'],
 ['사골국물',
  '다진마늘',
  '숙주',
  '썰은 표고버섯',
  '계란',
  '땅콩',
  '썰은',
  '고추',
  '두반장',
  '땅콩버터',
  '굴소스']]


- 84번 : 돔베고기....ㅠ
- 95번 : 음.........

> 세트구성인 경우 구성정보만 하나만 들어갔음. 추후 수정 필요..